# Projet N°4: Cheminée solaire
### Valentin Deumier & Augustin Ferrey

### Partie 1: Etude thermique simplifiée

![schéma_cheminée](schéma_cheminée.png)

On modélise le système par un modèle en quatre zones distinctes à température uniforme: 
* La zone d’air de la serre (asm) à $T_{asm}$
* La zone d’air de la cheminée (acm) à $T_{acm}$
* La zone de paroi du vitrage de la serre (pvm) à $T_{pvm}$
* La zone de paroi du sol de la serre (psm) à $T_{psm}$ 

Hypothèses supplémentaires : 
* L'air est transparent
* Comme le flux solaire est majoritairement dans le visible, le verre est considéré comme un corps gris avec $\epsilon = \alpha = 0.05$
* On néglige le facteur de forme entre le vitrage et la cheminée, et comme le verre est transparent, il émet dans tout son volume, et on suppose que la moitié de ce qu'il émet atteint le sol, et que l'autre moitié repart vers le ciel.

### Question 1.1

#### Bilan de flux surfacique sur le vitrage:

$$ \phi_{entrant} = \alpha \phi + \alpha \sigma T_{psm}^4 $$
$$ \phi_{sortant} = h_{es} (T_{pvm} - T_{ext}) + h_{cv} (T_{pvm} - T_{asm}) + \alpha \sigma T_{pvm}^4 $$

d'où:

$$ (1) \qquad \alpha \phi = h_{es} (T_{pvm} - T_{ext}) + h_{cv} (T_{pvm} - T_{asm}) + \alpha \sigma (T_{pvm}^4 - T_{psm}^4) $$


#### Bilan de flux surfacique sur le sol:

$$ \phi_{entrant} = \tau \phi + {1 \over 2} \alpha \sigma T_{pvm}^4 $$
$$ \phi_{sortant} = h_{cs} (T_{psm} - T_{asm}) + \sigma T_{psm}^4 $$

d'où:

$$ (2) \qquad \tau \phi = h_{cs} (T_{psm} - T_{asm}) + \sigma (T_{psm}^4 - {1 \over 2} \alpha T_{pvm}^4) $$


#### Bilan enthalpique sur l'air dans la serre:

$$ (3) \qquad q_{m0} (c_p (T_{ass} - T_{ase}) + {1 \over 2}(v_1^2 - v_{1'}^2)) = h_{cv} A_s (T_{pvm} - T_{asm}) + h_{cs} A_s (T_{psm} - T_{asm}) $$

où on a prit $c_p = 1009 J/kg/K$, $v_i = {q_{m0} \over \rho S_i}$ et $\rho = 1.184 kg/m^3$.


#### Bilan enthalpique sur l'air dans la cheminée:

$$ (4) \qquad q_{m0} (c_p (T_{acs} - T_{ace}) + g H_c) = KS (T_{ext} - T_{acm}) $$

avec $ {1 \over KS} = {1 \over h_{ec} A_{ce}} + {\ln({D_{ce} \over D_{ci}}) \over {2 \pi \lambda_b H_c}} + {1 \over h_{cc} A_{ci}}$

#### Bilan enthalpique sur tout l'air:

$$ (5) \qquad q_{m0} (c_p (T_{acs} - T_{ase}) + {1 \over 2}(v_2^2 - v_{1'}^2) + g H_c) = h_{cv} A_s (T_{pvm} - T_{asm}) + h_{cs} A_s (T_{psm} - T_{asm}) + KS (T_{ext} - T_{acm}) $$

Comme on a $ T_{ass} = T_{ace} $, $T_{asm} = 0.5 (T_{ase} + T_{ass})$ et $T_{acm} = 0.5 (T_{ace} + T_{acs})$, on a bien 5 équations pour trouver 5 inconnues.

In [8]:
from scipy.optimize import root
import numpy as np

# Parameters

sigma = 5.67e-8
alpha = 0.05
tau = 0.85
phi = 600
rho = 1.184

Hc = 1000
Hs = 30
Dci = 100
Dce = 101
Ds = 7000

As = 0.25*3.14*(Ds**2 - Dce**2)
Ace = 3.14*Dce*(Hc - Hs)
Aci = 3.14*Dci*(Hc - Hs)
Ac = 3.14*(Dci/2)**2

hes = 19
hec = 40
hcv = 11
hcs = 10
hcc = 12
lambdab = 2.3
KS = 1/(1/hec*Ace + 1/hcc*Aci + np.log(Dce/Dci)/(2*3.14*lambdab*Hc))
cp = 1009

Text = 298

g = 9.81
qm0 = 2*10**5
v11 = qm0/(rho*3.14*Ds*Hs)
v1 = qm0/(rho*3.14*Dce*Hs)
v2 = qm0/(rho*Ac)

# Equations

def equations(p):

    x, y, z, t, s = p   # x = Tase, y = Tass, z = Tacs, t = Tpvm, s = Tpsm

    tasm = (x + y)/2
    tacm = (y + z)/2

    eq1 = alpha*phi - hes*(t - Text) - hcv*(t - tasm) - alpha*sigma*(t**4 - s**4)  #Bilan sur le vitrage
    eq2 = tau*phi - hcs*(s - tasm) - sigma*(s**4 - 0.5*alpha*t**4)                 #Bilan sur le sol
    eq3 = qm0*(cp*(y - x) + 0.5*(v1**2 - v11**2)) - hcv*As*(t - tasm) - hcs*As*(s - tasm) #Bilan sur l'air asm
    eq4 = qm0*(cp*(z - y) + g*Hc) - KS*(Text - tacm) #Bilan sur l'air acm
    eq5 = qm0*(cp*(z - x) + 0.5*(v2**2 - v11**2) + g*Hc) - hcv*As*(t - tasm) - hcs*As*(s - tasm) - KS*(Text - tacm) #Bilan sur l'air total

    return [eq1, eq2, eq3, eq4, eq5]

# Solve
print(root(equations, [300, 300, 300, 300, 300]))

"""
x, y, z, t, s = fsolve(equations, [300, 300, 300, 300, 300])
print('Tase =', x)
print('Tass =', y)
print('Tacs =', z)
print('Tpvm =', t)
print('Tpsm =', s)
print('Tasm =', (x + y)/2)
print('Tacm =', (y + z)/2)
"""

 message: The iteration is not making good progress, as measured by the 
            improvement from the last ten iterations.
 success: False
  status: 5
     fun: [-3.264e+07 -6.261e+06 -5.022e+06 -4.939e+06  4.821e+06]
       x: [-4.382e+06  1.455e+06  1.455e+06 -8.776e+03 -9.323e+02]
    nfev: 100
    fjac: [[-1.925e-08 -1.750e-08 ...  0.000e+00 -7.071e-01]
           [-3.335e-08 -3.032e-08 ... -8.165e-01 -4.082e-01]
           ...
           [-6.762e-01  7.352e-01 ...  2.712e-02 -2.712e-02]
           [-5.859e-02 -1.172e-01 ...  5.724e-01 -5.724e-01]]
       r: [-2.857e+08 -7.138e+08 ...  1.045e+02 -1.720e+01]
     qtf: [-2.168e+05  1.124e+04 -2.920e+07  1.707e+07 -5.814e+06]


"\nx, y, z, t, s = fsolve(equations, [300, 300, 300, 300, 300])\nprint('Tase =', x)\nprint('Tass =', y)\nprint('Tacs =', z)\nprint('Tpvm =', t)\nprint('Tpsm =', s)\nprint('Tasm =', (x + y)/2)\nprint('Tacm =', (y + z)/2)\n"